# Problema térmico en 2D

En esta sección veremos algunos ejemplos para resolver el problema térmico en 2D en estado estacionario. Veremos el ejemplo en tres pasos: 1) generaremos la geometría y la malla con Gmsh y la importaremos a FEniCS, 2) luego resolveremos un caso del problema cilíndrico y lo compararemos con la solución teórica, y 3) resolveremos el problema no lineal cuando la conductividad térmica de las capas depende de la temperatura.

## Geometría y malla con Gmsh 

La geometría que deseamos resolver es sencilla, imaginemos una tubería lo suficientemente larga, con una temperatura interna ($T_{i}$), temperatura externa ($T_{e}$), y dos capas (por ejemplo: aislante y metal) con diferentes conductividades térmicas (ver la figura).

[<img src="figura2.png" width="200"/>](figura2.png)

Esto lo podemos **desarrollar en Gmsh** de forma parametrizada (en la carpeta [ejemplos](https://github.com/rirastorza/Intro2FEM/tree/master/ejemplos) ver el archivo: mallaejemplo11.geo). Aquí desglosaremos un poco este archivo (cuidado! no lo pueden correr directamente en jupyter notebook, ejecútelo aparte).

Primero definimos los parámetros (los radios) y luego los puntos parametrizados.

In [ ]:
gridsize = 0.1e-3;
r1 = 0.75e-3;
r2 = 0.95e-3;
R = 3.8e-3;
Point(1) = {3.5e-3, 3.5e-3, 0, gridsize};
Point(2) = {-r1+3.5e-3, 3.5e-3, 0, gridsize};
Point(3) = {r1+3.5e-3, 3.5e-3, 0, gridsize};
Point(4) = {-r2+3.5e-3, 3.5e-3, 0, gridsize};
Point(5) = {r2+3.5e-3, 3.5e-3, 0, gridsize};
Point(6) = {-R+3.5e-3, 3.5e-3, 0, gridsize};
Point(7) = {R+3.5e-3, 3.5e-3, 0, gridsize};

Luego las líneas circulares que unen esos puntos.

In [ ]:
Circle(1) = {2, 1, 3};
Circle(2) = {3, 1, 2};
Circle(3) = {4, 1, 5};
Circle(4) = {5, 1, 4};
Circle(5) = {6, 1, 7};
Circle(6) = {7, 1, 6};

Luego unimos esas líneas y creamos los lineloops para definir las superficies

In [ ]:
Line Loop(7) = {1,2};
Line Loop(8) = {3,4};
Line Loop(9) = {5,6};
Plane Surface(9) = {8,7};
Plane Surface(10) = {9, 8};

Finalmente, definimos (les ponemos un tag) las líneas y superficies físicas.

In [ ]:
Physical Line(10) = {1,2};
Physical Line(20) = {5,6};
Physical Surface(1) = {10};
Physical Surface(2) = {9};

Se puede colocar también la función: *Mesh.CharacteristicLengthMax = 0.2e-3;* que permite setear el tamaño máximo de la malla. Luego con la instrucción:

In [3]:
%%bash
gmsh -2 mallaejemplo11.geo -format msh2

Info    : Running 'gmsh -2 mallaejemplo11.geo -format msh2' [Gmsh 4.8.4, 1 node, max. 1 thread]
Info    : Started on Mon Sep  4 08:48:11 2023
Info    : Reading 'mallaejemplo11.geo'...
Info    : Done reading 'mallaejemplo11.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 20%] Meshing curve 2 (Circle)
Info    : [ 40%] Meshing curve 3 (Circle)
Info    : [ 50%] Meshing curve 4 (Circle)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : [ 90%] Meshing curve 6 (Circle)
Info    : Done meshing 1D (Wall 0.00100157s, CPU 0.001003s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 9 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 10 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0408642s, CPU 0.040847s)
Info    : 1379 nodes 2793 elements
Info    : Writing 'mallaejemplo11.msh'...
Info    : Done writing 'mallaejemplo11.msh'
Info    : Stopped on Mon Sep  4 08:48:11 2023 (From start: Wall 0.0490779s, CPU 0.100301s)


Como se ve, esto genera un archivo del mismo nombre pero con la extensión .msh. Lo debemos importar a FEniCS, una forma de hacerlo es transformando la malla a [formato xml](https://es.wikipedia.org/wiki/Extensible_Markup_Language). Para esto, los mismos desarrolladores de FEniCS tienen una función que permite hacer este procedimiento se denomina [*dolfin-convert*](https://people.sc.fsu.edu/~jburkardt/py_src/dolfin-convert/dolfin-convert.html). Ejecutando en consola:

In [4]:
%%bash
dolfin-convert mallaejemplo11.msh mallaejemplo11.xml

Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Expecting 1378 vertices
Found all vertices
Expecting 2612 cells
Found all cells
Conversion done


Se generan tres archivos: mallaejemplo11.xml, mallaejemplo11_physical_region.xml, y mallaejemplo11_facet_region.xml, que contienen la malla, las marcas de las regiones físicas, y las marcas de las fronteras o bordes. En el script *ejemplo11.py* hemos escrito todas estas instrucciones utilizando la herramienta de Python *os* que toma el sistema operativo y ejecuta los comandos que se le pasan como string. Por ejemplo: la primer instrucción por consola *gmsh -2 mallaejemplo11.geo* la podemos escribir en un script de Python como: 

*string = "gmsh -2 mallaejemplo11.geo"*

*os.system(string)*

Luego, siguiendo con el script de Python, ya podemos importar la malla, subdominios, y bordes como sigue:

In [5]:
from fenics import *

mesh = Mesh("mallaejemplo11.xml");
subdomains = MeshFunction('size_t',mesh,"mallaejemplo11_physical_region.xml");
boundary_markers = MeshFunction('size_t',mesh,"mallaejemplo11_facet_region.xml");

## Resolución de cilindro de dos capas

La **solución teórica** del problema presentado se puede ver como un problema de dos resistencias en serie (las dos capas). Como el problema tiene simetría cilíndrica entonces debemos resolver las ecuaciones de manera teórica en coordenadas cilíndricas. No repetiremos este cálculo aquí, simplemente tomaremos la solución del [libro de Kreith y otros](https://books.google.com.ar/books/about/Principles_of_Heat_Transfer.html?id=1hVSQBNvr74C&redir_esc=y) (ver página 82). En este caso, podemos encontrar una resistencia equivalente (por unidad de longitud) que tiene la forma:

$$R = \frac{ln(r_{2}/r_{1})}{2 \pi k_{1}} + \frac{ln(r_{3}/r_{2})}{2 \pi k_{2}} \tag{1}$$

Por lo que la tasa de flujo de calor por unidad de longitud ($\frac{q}{L}$) de la tubería será:

$$\frac{q}{L} = \frac{\Delta T}{R} \tag{2}$$

Tomando por ejemplo: $\Delta T = T_{i}-T_{e} = 314,15-310,15 = 4,0  $ K y $k_{1}= 10,0$  W/mK y $k_{2} = 400,0 $ W/mK, se tiene como resultado: $\frac{q}{L} = 927,24$  W/m.

### Formulación variacional en FEniCS

Comparemos este problema con nuestro script en FEniCS. Para esto debemos obtener la formulación variacional con condiciones de borde de Dirichlet, como ya lo hemos hecho antes.

$$ \left \{ \begin{array}{l} -\left(\nabla \cdot k \nabla T\right)=f \ \ \text{ para } \ \ x\in \Omega \\   T= 314,15 K  \ \ \text{condición de Dirichlet radio interno}  \ \ \partial \Omega_{1}\\   T= 310,15 K  \ \ \text{condición de Dirichlet radio externo} \ \ \partial \Omega_{2} \end{array} \right.$$

Hemos considerado sin fuente de calor $f = 0$. Aquí los bordes $\partial \Omega_{1}$ y $\partial \Omega_{2}$ serán las circunferencias internas y externas marcadas con 10 y 20, respectivamente (en el archivo .geo). Considerando la fórmula de Green, como vimos en el [tutorial anterior](https://github.com/rirastorza/Intro2FEM/blob/master/Elementos_finitos_en_2D/fem2D.ipynb), nos queda:

$$\int_{\Omega}k\nabla T \ \cdot \nabla v  da - \int_{\partial \Omega} v k\nabla T \cdot \overrightarrow{n}  ds=\int_{\Omega} f \ v \ da. \tag{3}$$

Como las condiciones de borde son ambas Dirichlet, entonces el segundo término de la izquierda lo podemos anular eligiendo convenientemente la función de prueba. Entonces queda:

$$\int_{\Omega}k\nabla T \ \cdot \nabla v  da =\int_{\Omega} f \ v \ da. \tag{4}$$

Esto lo podemos escribir directamente en FEniCS. Cuidado que aquí la conductividad térmica $k(x,y)$ es **función de la posición** entonces debemos decirle esto a FEniCS.

In [6]:
tol = 1E-14
#Constantes termicas
k_0 = Constant(10.0)
k_1 = Constant(400.0)

#Fuente, Temperatura variable
Tb = 310.15 #En Kelvin
DeltaT = 4.0

V = FunctionSpace(mesh, 'CG', 2)

#Defino condiciones de contorno por medio del archivo xml
bx0 = DirichletBC(V, Constant(Tb+DeltaT), boundary_markers,10)
bx1 = DirichletBC(V, Constant(Tb), boundary_markers, 20)
bcs = [bx0,bx1]

class K(UserExpression):
    def __init__(self, subdomains, k_0, k_1, **kwargs):
        super().__init__(**kwargs)
        self.subdomains = subdomains
        self.k_0 = k_0
        self.k_1 = k_1        
    def eval_cell(self, values, x, cell):
        if self.subdomains[cell.index] == 2:#capa interna del cilindro
            values[0] = self.k_0
        else:#capa externa del cilindro
            values[0] = self.k_1

kappa = K(subdomains, k_0, k_1, degree=2)


Note que aquí ya le hemos pasado los marcadores definidos en Gmsh, tanto a las condiciones de borde (10 y 20) como a la función que define la conductividad térmica (1 y 2).

Ahora si, escribmos la formulación variacional. Pero la vamos a escribir así:

$$F = \int_{\Omega}k\nabla T \ \cdot \nabla v  da - \int_{\Omega} f \ v \ da = 0 \tag{5}$$

In [7]:
u = Function(V)  
v = TestFunction(V)
f = Constant(0.00)

dx = dx(subdomain_data=subdomains)
ds = ds(subdomain_data=boundary_markers)

F = kappa*dot(grad(u), grad(v))*dx-f*v*dx

Ahora resolvemos. Noten que aquí no lo hicimos con la forma bilineal y lineal, lo implementa internamente FEniCS.

In [8]:
solve(F == 0, u, bcs)

No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 3.198e+06 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 8.417e-09 (tol = 1.000e-10) r (rel) = 2.632e-15 (tol = 1.000e-09)
  Newton solver finished in 1 iterations and 1 linear solver iterations.


Una vez que tenemos la solución, podemos calcular la tasa de flujo de calor por unidad de longitud así la comparamos con el resultado teórico. Para esto, primero debemos seleccionar la superficie (en este caso borde) donde queremos estimar el flujo, en este caso está etiquetada con 20. Note que también tenemos que conseguir la dirección normal para poder calcular lo siguiente:

$$\frac{q}{L} = -\int_{\partial \Omega_{2}} k\nabla T \cdot \overrightarrow{n} \ \ ds $$

In [9]:
#Cálculo del flujo
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)
n = FacetNormal(mesh)
flux = -(kappa)*dot(grad(u),n)*ds(20)
total_flux = assemble(flux)
print('Flujo numerico:',total_flux)

Flujo numerico: 924.3469714906935


Vemos que la diferencia es muy poca, se puede reducir haciendo una malla más fina.

## Problema con conductividad dependiente de la temperatura

El definir la formulación variacional de la forma $F = 0$ nos permite también utilizar este código para resolver problemas en los que la conductividad térmica depende de la temperatura. Es decir:

$$ \left \{ \begin{array}{l} -\left(\nabla \cdot k\left(T\right) \nabla T\right)=f \ \ \text{ para } \ \ x\in \Omega \\   T= 314,15 K  \ \ \text{condición de Dirichlet radio interno}  \ \ \partial \Omega_{1}\\   T= 310,15 K  \ \ \text{condición de Dirichlet radio externo} \ \ \partial \Omega_{2} \end{array} \right.$$

Ahora la conductividad dependerá de la posición y de la temperatura, es decir, la misma función que quiero computar. Por ejemplo, un función de temperatura que se utiliza mucho es la aproximación lineal en un rango no muy lejano de la temperatura de funcionamiento (digamos $T_{0}$). Supongamos que ésta aumenta el 2 % con el cambio de temperatura, es decir:

$$ k = k_{0}+\alpha k_{0} \left( T-T_{0} \right) = k_{0}\left( 1+\alpha \left( T-T_{0}\right) \right)$$

donde $\alpha = 0,02 \ \ K^{-1}$ y $k_{0}$ es la conductividad térmica a la temperatura $T_{0}$. En este tipo de problemas no voy a poder llegar a una expresión del tipo: $A\xi = b$. Por lo tanto, primero se **linealiza** y luego se resuelve **iterativamente**. En el medio, tiene que estimar el $F^{'}$. Por ahora no vamos a profundizar mucho en esto, solo diremos que  utiliza un algoritmo denominado **método inexacto de Newton** para resolverlo. Simplemente vamos a expresar la formulación variacional como sigue:

In [8]:
F = kappa*(1+alfa*(u-Constant(310.15)))*dot(grad(u), grad(v))*dx-f*v*dx

NameError: name 'alfa' is not defined

Esto último está resuelto en el *ejemplo 12.py*. Más adelante hablaremos de las soluciones de sistemas lineales y no lineales del tipo a los que aparecen con elementos fintos.